### Environment Setup

In [1]:
import os
import pandas as pd
import requests
from dotenv import load_dotenv

load_dotenv()
CMC_API_KEY = os.getenv("CMC_API_KEY")

if CMC_API_KEY:
    print("API_KEY found!")
else:
    print("API_KEY not found. Please set it in the .env file. See README.md for more info.")

API_KEY found!


### Data Fetching

In [2]:
# Fetch categories data from CoinMarketCap API
categories_url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/categories"
headers = {"Accepts": "application/json", "X-CMC_PRO_API_KEY": CMC_API_KEY}

response = requests.get(categories_url, headers=headers)

if response.status_code != 200:
    print(f"Error fetching categories: {response.status_code}")
    exit()

# Parse response data
categories_data = response.json()["data"]
print(f"Number of categories: {len(categories_data)}")

Number of categories: 259


### Data Processing

In [3]:
# Convert categories data to a DataFrame
categories_df = pd.DataFrame(categories_data)

# Rank market_cap_change and volume_change
categories_df["market_cap_rank"] = categories_df["market_cap_change"].rank(ascending=False)
categories_df["volume_rank"] = categories_df["volume_change"].rank(ascending=False)

# Combine ranks (lower is better)
categories_df["combined_rank"] = categories_df["market_cap_rank"] + categories_df["volume_rank"]

# Sort by combined rank (lower combined_rank indicates stronger performance)
categories_df = categories_df.sort_values(by="combined_rank")
print(categories_df[["name", "combined_rank", "market_cap_rank", "volume_rank"]])

# Get the top 1 category
top_category = categories_df.iloc[0]
print(f"Top category: {top_category['name']} (combined rank: {top_category['combined_rank']})")

                     name  combined_rank  market_cap_rank  volume_rank
255   NFTs & Collectibles           17.0             12.0          5.0
254   BNB Chain Ecosystem           25.0             13.0         12.0
137        HECO Ecosystem           28.0              6.0         22.0
243       Smart Contracts           55.0             11.0         44.0
198      Content Creation           57.0             18.0         39.0
..                    ...            ...              ...          ...
51          Reddit Points          479.0            225.0        254.0
244               Scaling          483.0            254.0        229.0
99   Kommunitas Launchpad          484.0            242.0        242.0
180            DeFi Index          487.0            237.0        250.0
230    Yearn Partnerships          490.0            241.0        249.0

[259 rows x 4 columns]
Top category: NFTs & Collectibles (combined rank: 17.0)


In [4]:
# Fetch coins in the top category
category_id = top_category["id"]
category_name = top_category["name"]

coins_url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/category"
headers = {"Accepts": "application/json", "X-CMC_PRO_API_KEY": CMC_API_KEY}
params = {"id": category_id}

response = requests.get(coins_url, headers=headers, params=params)

# Handle API response
if response.status_code != 200:
    print(f"Error: {response.status_code}, Message: {response.text}")
    exit()

data = response.json().get("data", {}).get("coins", [])
if not data:
    print(f"No coins found in category '{category_name}'.")
    exit()

# Process coins data
coins_df = pd.DataFrame(data)
if "quote" in coins_df.columns:
    coins_df["market_cap"] = coins_df["quote"].apply(
        lambda x: x.get("USD", {}).get("market_cap") if isinstance(x, dict) else None
    )

# Convert market_cap to numeric and sort
coins_df["market_cap"] = pd.to_numeric(coins_df["market_cap"], errors="coerce")
top_coins = coins_df.sort_values(by="market_cap", ascending=False).head(5)

# Print top 5 coins
print(f"Top 5 coins in category '{category_name}':")
print(top_coins[["name", "market_cap"]])

Top 5 coins in category 'NFTs & Collectibles':
                                    name    market_cap
0                                 Render  4.473849e+09
1  Artificial Superintelligence Alliance  3.819648e+09
2                                 Stacks  2.690270e+09
3                         Pudgy Penguins  2.621920e+09
4                              Immutable  2.483324e+09
